# Soil Classification Using ResNet18

This Jupyter notebook implements a binary image classification model to distinguish soil images from non-soil images (CIFAR-10 and MNIST datasets). The model uses a pre-trained ResNet18 with PyTorch, fine-tuned for the task. The code includes data loading, preprocessing, model training, and generating predictions for a test set.


## 1. Import Libraries and Dependencies

Importing necessary libraries for data handling, model building, and image processing.

In [18]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import warnings

warnings.filterwarnings('ignore')


## 2. Configuration Settings

Define hyperparameters and paths for the experiment.

### 2.1 Hyperparameters

In [4]:
BATCH_SIZE = 32
EPOCHS = 10
LR = 1e-4
IMG_SIZE = 224
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### 2.2 File Paths

In [7]:
soil_root = '../data/soil_competition-2025'  # Soil dataset path
cifar_root = '../data/cifar10/test'  # CIFAR-10 dataset path
mnist_root = '../data/mnist_png/testing'  # MNIST dataset path

## 3. Data Preparation

Load and preprocess the soil, CIFAR-10, and MNIST datasets, merging them into a single DataFrame.

### 3.1 Load Soil Data (Label = 1)

In [8]:
df_soil = pd.read_csv(os.path.join(soil_root, 'train_labels.csv'))
df_soil['path'] = df_soil['image_id'].apply(lambda x: os.path.join(soil_root, 'train', x))
df_soil['label'] = 1  # Soil images are labeled as 1

### 3.2 Load CIFAR-10 Data (Label = 0, 120 samples per class)


In [9]:
cifar_data = []
for cls in os.listdir(cifar_root):
    cls_path = os.path.join(cifar_root, cls)
    imgs = os.listdir(cls_path)[:120]  # Limit to 120 images per class
    for img in imgs:
        cifar_data.append({
            'image_id': img,
            'path': os.path.join(cls_path, img),
            'label': 0  # Non-soil images are labeled as 0
        })
df_cifar = pd.DataFrame(cifar_data)

### 3.3 Load MNIST Data (Label = 0, 80 samples per class)


In [10]:
mnist_data = []
for cls in os.listdir(mnist_root):
    cls_path = os.path.join(mnist_root, cls)
    imgs = os.listdir(cls_path)[:80]  # Limit to 80 images per class
    for img in imgs:
        mnist_data.append({
            'image_id': img,
            'path': os.path.join(cls_path, img),
            'label': 0  # Non-soil images are labeled as 0
        })
df_mnist = pd.DataFrame(mnist_data)

### 3.4 Merge and Split Data


In [11]:
df_all = pd.concat([df_soil[['image_id', 'path', 'label']], df_cifar, df_mnist], ignore_index=True)
df_train, df_val = train_test_split(df_all, test_size=0.2, stratify=df_all['label'], random_state=42)

## 4. Custom Dataset and DataLoaders

Define a custom dataset class and set up data transformations and DataLoaders.

### 4.1 Custom Dataset Class


In [12]:
class CombinedDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image = Image.open(self.df.iloc[idx]['path']).convert("RGB")
        label = self.df.iloc[idx]['label']
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label, dtype=torch.float32)

### 4.2 Data Transformations


In [13]:
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),  # Data augmentation
    transforms.RandomRotation(20),      # Data augmentation
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # Data augmentation
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet normalization
])

test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet normalization
])

### 4.3 DataLoaders


In [14]:
train_ds = CombinedDataset(df_train, transform=train_transform)
val_ds = CombinedDataset(df_val, transform=test_transform)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)

## 5. Model Setup

Initialize and configure the ResNet18 model for binary classification.

In [19]:
model = models.resnet18(pretrained=True)  # Use pre-trained weights
model.fc = nn.Linear(model.fc.in_features, 1)  # Modify final layer for binary output
model = model.to(DEVICE)

criterion = nn.BCEWithLogitsLoss()  # Loss function for binary classification
optimizer = torch.optim.Adam(model.parameters(), lr=LR)  # Adam optimizer
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2)  # Learning rate scheduler

## 6. Training Loop

Train the model and evaluate on the validation set for each epoch.

In [ ]:
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE).unsqueeze(1)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_train_loss = total_loss / len(train_loader)

    # Validation
    model.eval()
    preds, true = [], []
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(DEVICE)
            outputs = torch.sigmoid(model(images)).cpu().numpy()
            preds.extend(outputs)
            true.extend(labels.numpy())
    preds = [1 if p > 0.5 else 0 for p in preds]
    acc = accuracy_score(true, preds)
    scheduler.step(1 - acc)
    print(f"Epoch {epoch+1}: Loss {avg_train_loss:.4f}, Val Acc {acc:.4f}")

## 7. Generate Predictions for Test Set

Load test data and generate predictions for submission.

In [ ]:
test_root = '../data/soil_competition-2025/test'
test_ids = pd.read_csv('../data/soil_competition-2025/test_ids.csv')

model.eval()
preds = []

for img_name in test_ids['image_id']:
    img_path = os.path.join(test_root, img_name)
    img = Image.open(img_path).convert("RGB")
    img = test_transform(img).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        output = torch.sigmoid(model(img)).item()
        pred = 1 if output > 0.5 else 0
        preds.append(pred)

# Save predictions to CSV
submission = pd.DataFrame({
    'image_id': test_ids['image_id'],
    'label': preds
})
submission.to_csv('submission.csv', index=False)
print("✅ submission.csv saved")